## Data Ingestion

In [1]:
import os
import sys

In [2]:
%pwd

'/Users/dhananjaychaturvedi/Desktop/Nikita/collab_projects/Text_translation_eng_hin/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/dhananjaychaturvedi/Desktop/Nikita/collab_projects/Text_translation_eng_hin'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from texttranslation.utils.common import read_yaml,create_directories
from texttranslation.constants import *

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])  

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        
        ) 

        return data_ingestion_config     

In [8]:
import os
import urllib.request as request
import zipfile
from texttranslation.logging import logger
from texttranslation.utils.common import get_size

Create Components

In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url= self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        

## Create Pipeline

In [10]:
try:
    config= ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-26 12:16:59,744: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-26 12:16:59,748: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-26 12:16:59,749: INFO: common: created directory at: artifacts]
[2023-08-26 12:16:59,750: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-26 12:17:00,883: INFO: 879570419: artifacts/data_ingestion/text_data.zip downloaded! with following info: 
Connection: close
Content-Length: 100997
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "897b738d09c23a07213f91ea0dec3520c1a85d1ee4e91fb7751ea0db3fff355c"
Last-Modified: Sat, 26 Aug 2023 06:47:04 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2D44:38AD5A:EECBD:15B3B2:64E99FE7
Accept-Ranges: bytes
Date: Sat, 26 Aug 2023 0